In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device = torch.device("cpu")
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax
import sklearn
from sklearn.metrics import log_loss, f1_score

def seed_all(seed_value):
    random.seed(seed_value) 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value) 
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) 
        torch.backends.cudnn.deterministic = True  
        torch.backends.cudnn.benchmark = False

seed_all(79)

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1660


In [2]:
evidence = pd.read_csv('evidence.csv',sep='\t')

In [3]:
evidence.head()

,ID,evidence
0,0,"On Obamacare, Republicans generally fall into ..."
1,1,President Donald Trump stands firmly by the di...
2,2,"In a recent post on Medium, Robby Mook -- Hill..."
3,3,"During the 2020 State of the Union address, Pr..."
4,4,Republican presidential nominee Donald Trump p...


In [4]:
fact_checking_train = pd.read_csv('fact_checking_train.csv',sep='\t')

In [5]:
fact_checking_train.head()

,ID,author,claim,label
0,0,Joe Biden,"Sanders’ “Medicare for All” plan ""would cost m...",false
1,1,Hillary Clinton,"McCain ""still thinks it's okay when women don'...",false
2,2,Facebook posts,Says a video shows Iranian rockets being launc...,false
3,3,Tom Barrett,"""No one on my staff has ever been charged with...",false
4,4,City of Atlanta,Tyler Perry’s plan to turn a majority of the f...,half-true


In [6]:
print('the number of authors',len(set(fact_checking_train['author'])))

the number of authors 4170


In [7]:
fact_checking_train['label'].value_counts()

false          4462
half-true      3171
barely-true    2980
mostly-true    2898
true           2256
pants-fire     2233
Name: label, dtype: int64

In [8]:
fact_checking_test = pd.read_csv('fact_checking_test.csv',sep='\t')
fact_checking_test['label']=0

In [9]:
fact_checking_test.head()

,ID,author,claim,label
0,18000,Philadelphia Daily News,If you don't buy cigarettes at your local supe...,0
1,18001,Donald Trump,"""A few months ago, it hit where West Virginia ...",0
2,18002,Eric Cantor,"Wayne Powell has a ""stated position of having ...",0
3,18003,Facebook posts,“CNN-NPR photojournalist Jade Sacker was embed...,0
4,18004,Viral image,Says Whoopi Goldberg kept her job and was appl...,0


In [10]:
label_2_v = {'false':0,'half-true':1,'barely-true':2,'mostly-true':3,'true':4,'pants-fire':5}
fact_checking_train['label'] = fact_checking_train['label'].map(label_2_v)
fact_checking_train.head()

,ID,author,claim,label
0,0,Joe Biden,"Sanders’ “Medicare for All” plan ""would cost m...",0
1,1,Hillary Clinton,"McCain ""still thinks it's okay when women don'...",0
2,2,Facebook posts,Says a video shows Iranian rockets being launc...,0
3,3,Tom Barrett,"""No one on my staff has ever been charged with...",0
4,4,City of Atlanta,Tyler Perry’s plan to turn a majority of the f...,1


In [11]:
fact_checking_train['author'].value_counts()

Facebook posts    1299
Donald Trump       871
Viral image        681
Bloggers           586
Barack Obama       520
                  ... 
Onward Ohio          1
Rob Bovett           1
Josh Burgin          1
Tom Corbett          1
Joel Klatt           1
Name: author, Length: 4170, dtype: int64

In [12]:
def claim_len(x):
    return len(x)
fact_checking_train['claim_len'] = fact_checking_train['claim'].apply(claim_len)

In [13]:
fact_checking_train['claim_len'].mean()

109.07155555555556

In [14]:
f1=sklearn.metrics.f1_score

In [15]:
from simpletransformers.classification import ClassificationModel

# model configuration
model_args = { 
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'manual_seed': 79,
    "silent": False, 
    'num_train_epochs': 2,
    'learning_rate': 2e-5,
    'fp16': False,
    'max_seq_length': 128,
#     'train_batch_size': 1,
}

# Preparing train data
train = fact_checking_train[['claim','label']].copy()
train.columns = ["text", "labels"]

test = fact_checking_test[['claim','label']].copy()
test.columns = ["text", "labels"]


In [16]:
%%time
torch.cuda.empty_cache()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=79)
err=[]
y_pred_tot=[]
for train_index, test_index in kf.split(train, train['labels']):
  train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
  model_rb = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=6,  args=model_args)
  model_rb.train_model(train1_trn, eval_df=train1_val)
  result, model_outputs, _ = model_rb.eval_model(train1_val, acc=sklearn.metrics.accuracy_score)
  print(f"Accuracy:{result['acc']}")    
  err.append((result['acc']))
  predictions, raw_outputs  = model_rb.predict(test['text'].values.tolist())
  y_pred_tot.append(raw_outputs)
print("Mean f1 score: ",np.mean(err))
np.save('y_pred_tot',y_pred_tot)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

  0%|          | 0/16200 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/2025 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/2025 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/225 [00:00<?, ?it/s]

Accuracy:0.33055555555555555


  0%|          | 0/1793 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

Mean f1 score:  0.326
Wall time: 4h 13min 21s


In [17]:
# from simpletransformers.classification import ClassificationModel, ClassificationArgs
# import logging

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

In [18]:
# Optional model configuration
# model_args = ClassificationArgs()
# model_args.use_early_stopping = True
# model_args.early_stopping_delta = 0.01
# model_args.early_stopping_metric = "mcc"
# model_args.early_stopping_metric_minimize = False
# model_args.early_stopping_patience = 5
# model_args.evaluate_during_training_steps = 1000

# model_args = ClassificationArgs(num_train_epochs=1)

# # Create a ClassificationModel
# model = ClassificationModel(
#     'bert',
#     'bert-base-cased',
#     num_labels=6,
#     args=model_args
# ) 

# # Preparing train data
# train_df = fact_checking_train[['claim','label']].copy()
# train_df.columns = ["text", "labels"]
# # Train the model
# model.train_model(train_df)

In [19]:
def softmax(x):
    orig_shape = x.shape

    if len(x.shape) > 1:
        # Matrix
        exp_minmax = lambda x: np.exp(x - np.max(x))
        denom = lambda x: 1.0 / np.sum(x)
        x = np.apply_along_axis(exp_minmax,1,x)
        denominator = np.apply_along_axis(denom,1,x) 
        
        if len(denominator.shape) == 1:
            denominator = denominator.reshape((denominator.shape[0],1))
        
        x = x * denominator
    else:
        # Vector
        x_max = np.max(x)
        x = x - x_max
        numerator = np.exp(x)
        denominator =  1.0 / np.sum(numerator)
        x = numerator.dot(denominator)
    
    assert x.shape == orig_shape
    return x
sum(softmax(y_pred_tot[0])[0])

1.0

In [20]:
for i in range(10):
    if i == 0:
        res = softmax(y_pred_tot[i])/10
    else:
        res += softmax(y_pred_tot[i])/10

In [21]:
res = np.argmax(res, axis=1)

In [22]:
sub = pd.read_csv('submission.csv', sep='\t')

In [23]:
sub

,ID,label
0,18000,pants-fire
1,18001,barely-true
2,18002,true
3,18003,mostly-true
4,18004,false
...,...,...
1788,19788,mostly-true
1789,19789,true
1790,19790,false
1791,19791,false


In [24]:
sub['label'] = res

In [25]:
v_2_label = {0:'false',1:'half-true',2:'barely-true',3:'mostly-true',4:'true',5:'pants-fire'}
sub['label'] = sub['label'].map(v_2_label)

In [26]:
sub

,ID,label
0,18000,half-true
1,18001,mostly-true
2,18002,false
3,18003,false
4,18004,pants-fire
...,...,...
1788,19788,barely-true
1789,19789,pants-fire
1790,19790,mostly-true
1791,19791,mostly-true


In [27]:
sub.to_csv('sub_test.csv',index=None,sep='\t')

In [28]:
subtest =  pd.read_csv('./sub_test.csv', sep='\t')

In [29]:
subtest

,ID,label
0,18000,half-true
1,18001,mostly-true
2,18002,false
3,18003,false
4,18004,pants-fire
...,...,...
1788,19788,barely-true
1789,19789,pants-fire
1790,19790,mostly-true
1791,19791,mostly-true
